**Name:** Rafael Henrique Martello

**Date**: 05-09-2021


# Assignment 1 - Data Wrangling Skills

### Question 1 
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [1]:
import pandas as pd
import numpy as np

In [2]:
def answer_one():

    # --------------------------------------------------------------------------------------------------------------------------------------------- 
    # DataFrame 1 - energy
    # ---------------------------------------------------------------------------------------------------------------------------------------------   

    # removing the 17 first rows of header and the 38 last rows from the footer
    energy = pd.read_excel('Data_A1/Energy_Indicators.xls', skiprows=17, skipfooter=38) 
    
    # removing column 1 and 2
    energy = energy.drop('Unnamed: 0', axis=1)
    energy = energy.drop('Unnamed: 1', axis=1) 
    
    # Changing column names
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    ## Remove Numbers:
    energy['Country'] = energy['Country'].str.replace(r'\d*', '', regex=True)
    ## Remove parentheses and all data within:
    energy['Country'] = energy['Country'].str.replace(r" \(.*\)","", regex=True)

    #Changing country names:
    energy['Country'] = energy['Country'].replace(['Republic of Korea'],'South Korea')
    energy['Country'] = energy['Country'].replace(['United States of America'],'United States')
    energy['Country'] = energy['Country'].replace(['United Kingdom of Great Britain and Northern Ireland'],'United Kingdom')
    energy['Country'] = energy['Country'].replace(['China, Hong Kong Special Administrative Region'],'Hong Kong')

    # Converting Energy Supply to gigajoules
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    ## Removing non numerical elements from the dataframe:
    energy['Energy Supply'] = pd.to_numeric(energy['Energy Supply'], errors='coerce')
    energy['Energy Supply per Capita'] = pd.to_numeric(energy['Energy Supply per Capita'], errors='coerce')
    energy['% Renewable'] = pd.to_numeric(energy['% Renewable'], errors='coerce')
    energy.set_index(['Country'],inplace=True)
    
    # --------------------------------------------------------------------------------------------------------------------------------------------- 
    # DataFrame 2 - GDP
    # ---------------------------------------------------------------------------------------------------------------------------------------------   
    
    GDP = pd.read_csv('Data_A1/world_bank.csv',sep=',', skiprows=4)
    
    GDP = GDP[['Country Name','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']]

      #Changing country names:
    GDP['Country Name'] = GDP['Country Name'].replace(['Korea, Rep.'],'South Korea')
    GDP['Country Name'] = GDP['Country Name'].replace(['Iran, Islamic Rep.'],'Iran')
    GDP['Country Name'] = GDP['Country Name'].replace(['Hong Kong SAR, China'],'Hong Kong')
    GDP.set_index(['Country Name'],inplace=True)
    

    # --------------------------------------------------------------------------------------------------------------------------------------------- 
    # DataFrame 3 - ScimEn
    # ---------------------------------------------------------------------------------------------------------------------------------------------   
    
    ScimEn = pd.read_excel('Data_A1/scimagojr-3.xlsx')
    ScimEn.set_index(['Country'],inplace=True)
    
    # --------------------------------------------------------------------------------------------------------------------------------------------- 
    # Joining the 3 dataframes:
    # ---------------------------------------------------------------------------------------------------------------------------------------------   
    
    df1 = pd.merge(pd.merge(ScimEn,energy,left_on='Country',right_index=True,how ='outer'),GDP,left_on='Country',right_on='Country Name',how ='outer')
    df = df1[['Country','Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations','Citations per document', 'H index', 'Energy Supply','Energy Supply per Capita', '% Renewable', '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    df.set_index(['Country'],inplace =True)
    df = df.loc[df['Rank'].isin([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])]
    df = df.sort_values(by=['Rank'])

    return df


In [3]:
answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1.0,127050.0,126767.0,597237.0,411683.0,4.70,138.0,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2.0,96661.0,94747.0,792274.0,265436.0,8.20,230.0,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3.0,30504.0,30287.0,223024.0,61554.0,7.31,134.0,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4.0,20944.0,20357.0,206091.0,37874.0,9.84,139.0,7.920000e+09,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5.0,18534.0,18301.0,34266.0,12422.0,1.85,57.0,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6.0,17899.0,17620.0,215003.0,40930.0,12.01,149.0,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7.0,17027.0,16831.0,140566.0,27426.0,8.26,126.0,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8.0,15005.0,14841.0,128763.0,37209.0,8.58,115.0,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9.0,13153.0,12973.0,130632.0,28601.0,9.93,114.0,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [4]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [5]:
def answer_two():
    # --------------------------------------------------------------------------------------------------------------------------------------------- 
    # DataFrame 1 - energy
    # ---------------------------------------------------------------------------------------------------------------------------------------------   

    # removing the 17 first rows of header and the 38 last rows from the footer
    energy = pd.read_excel('Data_A1/Energy_Indicators.xls', skiprows=17, skipfooter=38) 
    
    # removing column 1 and 2
    energy = energy.drop('Unnamed: 0', axis=1)
    energy = energy.drop('Unnamed: 1', axis=1) 
    
    # Changing column names
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    ## Remove Numbers:
    energy['Country'] = energy['Country'].str.replace(r'\d*', '', regex=True)
    ## Remove parentheses and all data within:
    energy['Country'] = energy['Country'].str.replace(r" \(.*\)","", regex=True)

    #Changing country names:
    energy['Country'] = energy['Country'].replace(['Republic of Korea'],'South Korea')
    energy['Country'] = energy['Country'].replace(['United States of America'],'United States')
    energy['Country'] = energy['Country'].replace(['United Kingdom of Great Britain and Northern Ireland'],'United Kingdom')
    energy['Country'] = energy['Country'].replace(['China, Hong Kong Special Administrative Region'],'Hong Kong')

    energy.set_index(['Country'],inplace=True)
    
    # --------------------------------------------------------------------------------------------------------------------------------------------- 
    # DataFrame 2 - GDP
    # ---------------------------------------------------------------------------------------------------------------------------------------------   
    
    GDP = pd.read_csv('Data_A1/world_bank.csv',sep=',', skiprows=4)
    
    GDP = GDP[['Country Name','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']]

      #Changing country names:
    GDP['Country Name'] = GDP['Country Name'].replace(['Korea, Rep.'],'South Korea')
    GDP['Country Name'] = GDP['Country Name'].replace(['Iran, Islamic Rep.'],'Iran')
    GDP['Country Name'] = GDP['Country Name'].replace(['Hong Kong SAR, China'],'Hong Kong')    

    # --------------------------------------------------------------------------------------------------------------------------------------------- 
    # DataFrame 3 - ScimEn
    # ---------------------------------------------------------------------------------------------------------------------------------------------   
    
    ScimEn = pd.read_excel('Data_A1/scimagojr-3.xlsx')
    
    # --------------------------------------------------------------------------------------------------------------------------------------------- 
    # Avaliate the union - intersection:
    # ---------------------------------------------------------------------------------------------------------------------------------------------   

    #Create the union (outer join)
    union =  pd.merge(pd.merge(ScimEn,energy,left_on='Country',right_index=True,how ='outer'),GDP,left_on='Country',right_on='Country Name',how ='outer')
    
    #Create the intersection (outer inner)
    intersect =  pd.merge(pd.merge(ScimEn,energy,left_on='Country',right_index=True,how ='inner'),GDP,left_on='Country',right_on='Country Name',how ='inner')
    return len(union)-len(intersect)

In [6]:
answer_two()

158

### Question 3
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [7]:
def answer_three():
    df = answer_one()
    # calculate the bean from 2006 to 2015
    avgGDP = df[['2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1,skipna = True).sort_index(ascending = False)
    return (avgGDP)

In [8]:
answer_three()

Country
United States         1.536434e+13
United Kingdom        2.487907e+12
Spain                 1.418078e+12
South Korea           1.106715e+12
Russian Federation    1.565459e+12
Japan                 5.542208e+12
Italy                 2.120175e+12
Iran                  4.441558e+11
India                 1.769297e+12
Germany               3.493025e+12
France                2.681725e+12
China                 6.348609e+12
Canada                1.660647e+12
Brazil                2.189794e+12
Australia             1.164043e+12
dtype: float64

### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [9]:
def answer_four():
    df = answer_one()
    avgGDP = answer_three()
    df['avgGDP'] = avgGDP
    # sort avgGDP values
    df.sort_values(['avgGDP'],ascending = False,inplace=True)
    # for the 6th [5] do 2015 - 2006
    return df.iloc[5]['2015']-df.iloc[5]['2006']
    

In [10]:
answer_four()

246702696075.3999

### Question 5
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [11]:
def answer_five():
    df = answer_one()
    energy_mean = df['Energy Supply per Capita'].mean()
    return energy_mean

In [12]:
answer_five()

157.6

### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [13]:
def answer_six():
    df = answer_one()
    # using max to obtain the maximum % Renewable
    max_renew = df[df['% Renewable'] == max(df['% Renewable'])]
    
    # get the country name as index and the % 
    return (max_renew.index.tolist()[0],max_renew['% Renewable'].tolist()[0])

In [14]:
answer_six()

('Brazil', 69.64803)

### Question 7 
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [15]:
def answer_seven():
    df = answer_one()
    # calculate the citation ratio by = self-citations/citations
    df['Citation Ratio'] = df['Self-citations']/df['Citations']
    # calculate the max ratio
    max_ratio = df[df['Citation Ratio'] == max(df['Citation Ratio'])]
    
    # get country name and max_ratio
    return (max_ratio.index.tolist()[0],max_ratio['Citation Ratio'].tolist()[0])

In [16]:
answer_seven()

('China', 0.6893126179389422)

### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [17]:
def answer_eight():
    df = answer_one()

    # calculate population by Energy Supply divided by Energy per Capita
    df['Population'] = df['Energy Supply']/df['Energy Supply per Capita']
    # sort values
    df = df.sort_values(by=['Population'],ascending=False)

    # get 3th [2] country name and number of population
    return (df['Population'].index.tolist()[2], df['Population'].tolist()[2])

In [18]:
answer_eight()

('United States', 317615384.61538464)

### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*


In [19]:
def answer_nine():
    df = answer_one()

    # Calculate the number of population
    df['Population'] = df['Energy Supply']/df['Energy Supply per Capita']
    # create Citable docs per Capita for estimating the number of citable documents per person
    df['Citable docs per Capita'] = df['Citable documents'] / df['Population']
    # estimate correlation
    correlation = df['Citable docs per Capita'].corr(df['Energy Supply per Capita'], method='pearson')
    return correlation


In [20]:
answer_nine()

0.7940010435442946

### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [21]:
def answer_ten():
    df = answer_one()
    # calculate median
    median = df['% Renewable'].median() 
    # as the df ir already sorted by rank we just need to check data with the median
    df['HighRenew'] = [1 if x >= median else 0 for x in df['% Renewable']]
    return df['HighRenew']

In [22]:
answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Question 11
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [23]:
def answer_eleven():
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    df = answer_one()

    # calculate population by Energy Supply divided by Energy per Capita
    df['Population'] = df['Energy Supply']/df['Energy Supply per Capita'].astype(float)
    df['Population'] = np.float64(df['Population'])

    # rename index with continentdict
    df.rename(index=ContinentDict,inplace=True)
    df.reset_index(inplace = True)

    # declare functions
    functions = ['size', 'sum', 'mean', 'std']

    # Create the continent pupulation table
    Population_df = df[['Country','Population']].groupby('Country').agg(functions)
    return Population_df



In [24]:
answer_eleven()

Population                                          
                    size           sum          mean           std
Country                                                           
Asia                   5  2.898666e+09  5.797333e+08  6.790979e+08
Australia              1  2.331602e+07  2.331602e+07           NaN
Europe                 6  4.579297e+08  7.632161e+07  3.464767e+07
North America          2  3.528552e+08  1.764276e+08  1.996696e+08
South America          1  2.059153e+08  2.059153e+08           NaN

### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [25]:
def answer_twelve():
    df=answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    df = df.reset_index()
    # first we create the continent column
    df['Continent'] = [ContinentDict[country] for country in df['Country']]
    # usin pd.cut we separate the data in 5 bins 
    df['bins'] = pd.cut(df['% Renewable'],5)
    # we show the continents, bins and number os countries using size
    return df.groupby(['Continent','bins']).size()

In [26]:
answer_twelve()

Continent      bins            
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Australia      (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
               (42.701, 56.174]    0
               (56.174, 69.648]    0
North America  (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
South America  (2.212, 15.753]     0
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
dtype: int64

### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [27]:
def answer_thirteen():
    import locale
    import pandas as pd

    locale.setlocale(locale.LC_ALL, 'en_US.utf8')

    df = answer_one()

    # calculate population by Energy Supply divided by Energy per Capita
    df['Population'] = df['Energy Supply']/df['Energy Supply per Capita'].astype(float)
    
    map_str = []
    for i in df['Population']:
        # convert format
        map_str.append(locale.format_string('%.2f',i,grouping=True))
    df['PopEst'] = map_str
    return df['PopEst']


In [28]:
answer_thirteen()

Country
China                 1,367,645,161.29
United States           317,615,384.62
Japan                   127,409,395.97
United Kingdom           63,870,967.74
Russian Federation      143,500,000.00
Canada                   35,239,864.86
Germany                  80,369,696.97
India                 1,276,730,769.23
France                   63,837,349.40
South Korea              49,805,429.86
Italy                    59,908,256.88
Spain                    46,443,396.23
Iran                     77,075,630.25
Australia                23,316,017.32
Brazil                  205,915,254.24
Name: PopEst, dtype: object